In [1]:
# Day_02_06_slim.py
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [2]:
def use_variable():
    w1 = slim.variable('w_1', shape=[10])
    w2 = slim.model_variable('w_2', shape=[10])

    print(w1)
    print(w2)

    print(slim.get_variables())
    print(slim.get_model_variables())

In [3]:
def use_network():
    input = slim.variable('input', shape=[10, 10, 3, 5])

    with tf.variable_scope('c_1'):
        kernel = tf.Variable(tf.zeros([10, 10, 5, 5]))
        conv = tf.nn.conv2d(input, kernel, [1, 1, 1, 1], padding='SAME')

        bias = tf.Variable(tf.ones([5]))
        bias_add = tf.nn.bias_add(conv, bias)

        conv_1 = tf.nn.relu(bias_add)

    conv_2 = slim.conv2d(inputs=input, num_outputs=5, kernel_size=[3, 3])

    net_1  = tf.placeholder(tf.float32, [16, 32, 32, 256])
    with tf.variable_scope('c_2'):
        net_1 = slim.conv2d(net_1, 256, [3, 3], scope='conv_3_1')
        net_1 = slim.conv2d(net_1, 256, [3, 3], scope='conv_3_2')
        net_1 = slim.conv2d(net_1, 256, [3, 3], scope='conv_3_3')
        net_1 = slim.max_pool2d(net_1, [2, 2], scope='pool_3')

    # 반복문
    net_2 = tf.placeholder(tf.float32, [16, 32, 32, 256])
    with tf.variable_scope('c_3'):
        for i in range(3):
            net_2 = slim.conv2d(net_2, 256, [3, 3], scope='conv_3_'+ str(i))
        net_2 = slim.max_pool2d(net_2, [2, 2], scope='pool_3')

    # repeat
    net_3 = tf.placeholder(tf.float32, [16, 32, 32, 256])
    with tf.variable_scope('c_4'):
        net_3 = slim.repeat(net_3, 3, slim.conv2d, 256, [3, 3], scope='rep')
        # print(net_3)
        net_3 = slim.max_pool2d(net_3, [2, 2], scope='pool_3')

    # -------------------------------------------------------------------------- #
    net_4 = tf.placeholder(tf.float32, [16, 32, 32, 256])
    net_4 = slim.conv2d(net_4, 32, [3, 3], scope='conv_4_1')
    net_4 = slim.conv2d(net_4, 32, [5, 5], scope='conv_4_2')
    net_4 = slim.conv2d(net_4, 64, [3, 3], scope='conv_4_3')
    net_4 = slim.conv2d(net_4, 64, [5, 5], scope='conv_4_4')

    # stack
    binder = [(32, [3, 3]), (32, [5, 5]), (64, [3, 3]), (64, [5, 5])]
    net_5 = tf.placeholder(tf.float32, [16, 32, 32, 256])
    net_5 = slim.stack(net_5, slim.conv2d, binder, scope='stk')

    # -------------------------------------------------------------------------- #

In [4]:
def use_arg_scope():
    with tf.variable_scope('a_1'):
        net_1 = tf.placeholder(tf.float32, [16, 300, 300, 64])

        # regularizer는  weight값이  너무 커지지 않게 조절해주는것
        # L1 : 절대값
        # L2 : 제곱해주는것
        net_1 = slim.conv2d(inputs=net_1, num_outputs=64,
                            kernel_size=[11,11],
                            padding='SAME',
                            stride=4,
                            weights_initializer=tf.zeros_initializer(),
                            weights_regularizer=slim.l2_regularizer(0.005))
        net_1 = slim.conv2d(inputs=net_1, num_outputs=128,
                            kernel_size=[11,11],
                            padding='VALID',
                            weights_initializer=tf.zeros_initializer(),
                            weights_regularizer=slim.l2_regularizer(0.005))
        net_1 = slim.conv2d(inputs=net_1, num_outputs=256,
                            kernel_size=[11,11],
                            padding='SAME',
                            weights_initializer=tf.zeros_initializer(),
                            weights_regularizer=slim.l2_regularizer(0.005))

    # 공통 부분 추출
    with tf.variable_scope('a_2'):
        net_1 = tf.placeholder(tf.float32, [16, 300, 300, 64])

        padding = 'SAME'
        initializer = tf.zeros_initializer()
        regularizer = slim.l2_regularizer(0.005)


        net_1 = slim.conv2d(inputs=net_1, num_outputs=64,
                            kernel_size=[11, 11],
                            padding=padding,
                            stride=4,
                            weights_initializer=initializer,
                            weights_regularizer=regularizer)
        net_1 = slim.conv2d(inputs=net_1, num_outputs=256,
                            kernel_size=[11, 11],
                            padding='VALID',
                            weights_initializer=initializer,
                            weights_regularizer=regularizer)
        net_1 = slim.conv2d(inputs=net_1, num_outputs=128,
                            kernel_size=[11, 11],
                            padding=padding,
                            weights_initializer=initializer,
                            weights_regularizer=regularizer)

    # arg_scope 사용
    with tf.variable_scope('a_3'):
        net_1 = tf.placeholder(tf.float32, [16, 300, 300, 64])

        with slim.arg_scope([slim.conv2d],
                            padding='SAME',
                            weights_initializer = tf.zeros_initializer(),
                            weights_regularizer = slim.l2_regularizer(0.005)
                            ):

            net_1 = slim.conv2d(inputs=net_1, num_outputs=64,
                                kernel_size=[11, 11],
                                stride=4)
            net_1 = slim.conv2d(inputs=net_1, num_outputs=256,
                                kernel_size=[11, 11],
                                padding='VALID')
            net_1 = slim.conv2d(inputs=net_1, num_outputs=128,
                                kernel_size=[11, 11])

    # -------------------------------------------------------------------------- #
    # 중복
    with tf.variable_scope('a_4'):
        net_1 = tf.placeholder(tf.float32, [16, 300, 300, 64])

        # 공통 부분만 써야 한다.
        with slim.arg_scope([slim.conv2d, slim.fully_connected],
                            weights_initializer=tf.zeros_initializer(),
                            weights_regularizer=slim.l2_regularizer(0.005)
                            ):
            net_1 = slim.conv2d(inputs=net_1, num_outputs=64,
                                kernel_size=[11, 11],
                                padding='SAME',
                                stride=4)
            net_1 = slim.conv2d(inputs=net_1, num_outputs=256,
                                kernel_size=[11, 11],
                                padding='VALID')
            net_1 = slim.conv2d(inputs=net_1, num_outputs=128,
                                kernel_size=[11, 11],
                                padding='SAME')
            net_1 = slim.fully_connected(inputs=net_1,
                                         num_outputs=256,
                                         activation_fn=None
                                         )

    with tf.variable_scope('a_5'):
        net_1 = tf.placeholder(tf.float32, [16, 300, 300, 64])

        # 공통 부분만 써야 한다.
        with slim.arg_scope([slim.conv2d, slim.fully_connected],
                            weights_initializer=tf.zeros_initializer(),
                            weights_regularizer=slim.l2_regularizer(0.005)
                            ):
            with slim.arg_scope([slim.conv2d],
                                padding='SAME'):
                net_1 = slim.conv2d(inputs=net_1, num_outputs=64,
                                    kernel_size=[11, 11],
                                    stride=4)
                net_1 = slim.conv2d(inputs=net_1, num_outputs=256,
                                    kernel_size=[11, 11],
                                    padding='VALID')
                net_1 = slim.conv2d(inputs=net_1, num_outputs=128,
                                    kernel_size=[11, 11])
            net_1 = slim.fully_connected(inputs=net_1,
                                         num_outputs=256,
                                         activation_fn=None
                                         )

In [ ]:
# use_variable()
# use_network()
use_arg_scope()